#### Лабораторная работа 5. Метод потенциалов для решения транспортной задачи 
Реализовать метод потенциалов для решения транспортной задачи.

$----------------------------------------------$

In [253]:
import numpy as np

<u>Шаг 0.</u> Определение исходных данных задачи

In [254]:
a = np.array([100, 300, 300])
b = np.array([300, 200, 200])
c = np.array([[8, 4, 1], [8, 4, 3], [9, 7, 5]])

n = a.size
m = b.size
b_path = []

<u>Шаг 1.</u> Проверим задачу на открытость. При необходимости добавим фиктивных поставщиков или потребителей

In [255]:
a_sum = np.sum(a)
b_sum = np.sum(b)

if a_sum != b_sum:
    print("Задача открытая")
    if a_sum > b_sum:
        c = np.append(c, [np.zeros(n)], axis=0)
        b = np.append(b, (a_sum - b_sum))
    if a_sum < b_sum:
        zer = np.array([0])
        c = np.append(c, np.repeat(zer[np.newaxis,:], c.shape[0], axis=0), axis=1)
        a = np.append(a, b_sum - a_sum)
else:
    print("Задача закрытая")

n = a.size
m = b.size

Задача закрытая


<u>Шаг 2.</u> Сформируем начальный допустимый план $X$, используя для этого метод северо-западного угла.

In [256]:
a_copy = np.copy(a)
b_copy = np.copy(b)
x = np.zeros((m, n))
i = 0
j = 0

while i != n:
    if a_copy[i] != 0 and b_copy[j] != 0:
        if a_copy[i] > b_copy[j]:
            x[i][j] = b_copy[j]
            a_copy[i] -= b_copy[j]
            b_copy[j] = 0
            j += 1
        elif a_copy[i] < b_copy[j]:
            x[i][j] = a_copy[i]
            b_copy[j] -= a_copy[i]
            a_copy[i] = 0
            i += 1
        elif a_copy[i] == b_copy[j]:
            x[i][j] = a_copy[i]
            a_copy[i] = 0
            b_copy[j] = 0
            i += 1
            j += 1
    elif a_copy[i] != 0:
        i += 1
    elif b_copy[j] != 0:
        j += 1

print(x)

[[100.   0.   0.]
 [200. 100.   0.]
 [  0. 100. 200.]]


<u>Шаг 3.</u> Определим потенциалы для всех занятых ячеек. Первоначально определим потенциалы для клетки с наибольшей стоимостью перевозки

In [257]:
a_potential = np.zeros(n)
b_potential = np.zeros(m)
a_potential_flag = np.zeros(n)
b_potential_flag = np.zeros(m)

c_copy = np.zeros((m, n))

for i in range(0, n):
    for j in range(0, m):
        if x[j][i] != 0:
            c_copy[j][i] = c[j][i]

for i in b_path:
    c_copy[i[0]][i[1]] = c[i[0]][i[1]]
    
c_max = np.max(c_copy)

for i in range(0, n):
    for j in range(0, m):
        if c[j][i] == c_max and x[j][i] != 0:
            if c[j][i] % 2 == 0:
                a_potential[i] = c[j][i]/2
                b_potential[j] = c[j][i]/2
                a_potential_flag[i] = 1
                b_potential_flag[j] = 1
            else: 
                a_potential[i] = int(c[j][i]/2)
                b_potential[j] = int(c[j][i]/2) + 1
                a_potential_flag[i] = 1
                b_potential_flag[j] = 1
                
print(a_potential)
print(b_potential)

[4. 0. 0.]
[4. 4. 0.]


<u>Шаг 4.</u> Теперь найдем оставшиеся потенциалы

In [258]:
while(np.max(c_copy) != 0):
    for i in range(0, n):
        for j in range(0, m):
            if c_copy[j][i] > 0:
                if (a_potential[i] != 0 or a_potential_flag[i] != 0) and b_potential_flag[j] == 0:
                    b_potential[j] = c_copy[j][i] - a_potential[i]
                    b_potential_flag[j] = 1
                    c_copy[j][i] = -1
                elif (b_potential[j] != 0 or b_potential_flag[j] != 0) and a_potential_flag[i] == 0:
                    a_potential[i] = c_copy[j][i] - b_potential[j]
                    a_potential_flag[i] = 1
                    c_copy[j][i] = -1
                elif (a_potential[i] != 0 or a_potential_flag[i] != 0) and (b_potential[i] != 0 or b_potential_flag[i] != 0):
                    c_copy[j][i] = -1
                    
print(a_potential)
print(b_potential)

[ 4.  0. -2.]
[4. 4. 7.]


<u>Шаг 5.</u> Построим матрицу потенциалов для всех клеток

In [259]:
new_p = np.zeros(c.shape)
for i in range(len(a_potential)):
    for j in range(len(b_potential)):
        new_p[j][i] = a_potential[i] + b_potential[j]
        
new_p

array([[ 8.,  4.,  2.],
       [ 8.,  4.,  2.],
       [11.,  7.,  5.]])

<u>Шаг 6.</u> Найдем наиболее неоптимальную клетку (или убедимся, что таких не осталось)

In [260]:
min_elem = 1
out_coord = [0, 0]

for i in range(n):
    for j in range(m):
        if c[i][j] - new_p[i][j] < min_elem:
            out_coord[0] = i
            out_coord[1] = j
            min_elem = c[i][j] - new_p[i][j]

if min_elem == 1:
    print("План оптимальный")
    print(x)
    
x[out_coord[0]][out_coord[1]] = -1
start_point = tuple(out_coord)

<u>Шаг 7.</u> Найдем цикл с неоптимальной точкой, где все другие элементы были бы базисные. Для этого рекурсивно проверим точки вокруг найденой неоптимальной.

In [261]:
FLAG_END = False


def find_cycle(mat: list, pos: tuple, dir_row: bool = True, path: list = None) -> list:
    """ Построение цикла опорного плана транспортной задачи.
        При условии, что цикл существует и он единственный.
    """
    global FLAG_END

    if not path:
        path = [pos]
    ii, jj = pos
    # Собираем сначала соседей по строке, потом через раз на повороте.
    if dir_row:
        neighbors = [(ii, k) for k in range(len(mat[0])) if k != jj and mat[ii][k]]
    # Собираем соседей по столбцу на каждом втором повороте.
    else:
        neighbors = [(k, jj) for k in range(len(mat)) if k != ii and mat[k][jj]]
    # Если есть соседи:
    if neighbors:
        # Если стартовая точка есть в списке соседей - путь найден!
        if start_point in neighbors:
            FLAG_END = True
            return path
        else:
            # Меняем направление для поворота.
            dir_row = not dir_row
            # Перебираем всех соседей.
            for neighbor in neighbors:
                # Кладем очередную вершину в список пути.
                path.append(neighbor)
                # Запускаем рекурсию по ненулевым соседям.
                find_cycle(mat, neighbor, dir_row, path)
                # Если нашли путь дальше не перебираем.
                if FLAG_END:
                    break
                # Если путь не найден, то удаляем вершину из списка пути.
                else:
                    path.pop()
    return path
    
b_path = find_cycle(x, start_point)
x[out_coord[0]][out_coord[1]] = 0
FLAG_END = False
b_path

[(2, 0), (2, 1), (1, 1), (1, 0)]

<u>Шаг 8.</u> Создадим вектор знаков для найденного цикла

In [262]:
signs = []
i = 0

for i in range(len(b_path)):
    if i%2 == 0:
        signs.append('+')
    else:
        signs.append('-')
    i += 1
    
signs

['+', '-', '+', '-']

<u>Шаг 9.</u> Найдем наименьший элемент цикла со знаком '-'

In [263]:
min_cycle_elem = x[b_path[1][0]][b_path[1][1]]
min_elem_coord = (b_path[1][0], b_path[1][1])
for i in range(len(b_path)):
    if signs[i] == '-':
        if x[b_path[i][0]][b_path[i][1]] < min_cycle_elem:
            min_cycle_elem = x[b_path[i][0]][b_path[i][1]]
            min_elem_coord = (b_path[1][0], b_path[1][1])
            
min_cycle_elem

100.0

<u>Шаг 10.</u> Проведем преобразования транспортного плана (при этом удалим элемент с наименьшим значением из вектора пути) и переходим к шагу 3

In [264]:
for i in range(len(b_path)):
    if signs[i] == '-':
        x[b_path[i][0]][b_path[i][1]] = x[b_path[i][0]][b_path[i][1]] - min_cycle_elem
    elif signs[i] == '+':
        x[b_path[i][0]][b_path[i][1]] = x[b_path[i][0]][b_path[i][1]] + min_cycle_elem

b_path.remove(min_elem_coord)
x

array([[100.,   0.,   0.],
       [100., 200.,   0.],
       [100.,   0., 200.]])

In [265]:
while(True):
    #Шаг 3 ==========================================
    a_potential = np.zeros(n)
    b_potential = np.zeros(m)
    a_potential_flag = np.zeros(n)
    b_potential_flag = np.zeros(m)

    c_copy = np.zeros((m, n))

    for i in range(0, n):
        for j in range(0, m):
            if x[j][i] != 0:
                c_copy[j][i] = c[j][i]

    for i in b_path:
        c_copy[i[0]][i[1]] = c[i[0]][i[1]]
    
    c_max = np.max(c_copy)

    for i in range(0, n):
        for j in range(0, m):
            if c[j][i] == c_max and x[j][i] != 0:
                if c[j][i] % 2 == 0:
                    a_potential[i] = c[j][i]/2
                    b_potential[j] = c[j][i]/2
                    a_potential_flag[i] = 1
                    b_potential_flag[j] = 1
                else: 
                    a_potential[i] = int(c[j][i]/2)
                    b_potential[j] = int(c[j][i]/2) + 1
                    a_potential_flag[i] = 1
                    b_potential_flag[j] = 1

    #Шаг 4 ==========================================
    while(np.max(c_copy) != 0):
        for i in range(0, n):
            for j in range(0, m):
                if c_copy[j][i] > 0:
                    if (a_potential[i] != 0 or a_potential_flag[i] != 0) and b_potential_flag[j] == 0:
                        b_potential[j] = c_copy[j][i] - a_potential[i]
                        b_potential_flag[j] = 1
                        c_copy[j][i] = -1
                    elif (b_potential[j] != 0 or b_potential_flag[j] != 0) and a_potential_flag[i] == 0:
                        a_potential[i] = c_copy[j][i] - b_potential[j]
                        a_potential_flag[i] = 1
                        c_copy[j][i] = -1
                    elif (a_potential[i] != 0 or a_potential_flag[i] != 0) and (b_potential[i] != 0 or b_potential_flag[i] != 0):
                        c_copy[j][i] = -1
                    
    #Шаг 5 ==========================================
    new_p = np.zeros(c.shape)
    for i in range(len(a_potential)):
        for j in range(len(b_potential)):
            new_p[j][i] = a_potential[i] + b_potential[j]
            
    #Шаг 6 ==========================================
    min_elem = 0
    out_coord = [0, 0]
    for i in range(n):
        for j in range(m):
            if c[i][j] - new_p[i][j] < min_elem:
                out_coord[0] = i
                out_coord[1] = j
                min_elem = c[i][j] - new_p[i][j]
    if min_elem == 0:
        print("Оптимальный план: ")
        print(x)
        break
    
    x[out_coord[0]][out_coord[1]] = -1
    start_point = tuple(out_coord)

    #Шаг 7 ==========================================
    b_path = find_cycle(x, start_point, start_point, False)
    x[out_coord[0]][out_coord[1]] = 0
    FLAG_END = False
  
    #Шаг 8 ==========================================
    signs = []
    i = 0
    for i in range(len(b_path)):
        if i%2 == 0:
            signs.append('+')
        else:
            signs.append('-')
        i += 1
        
    #Шаг 9 ==========================================
    min_cycle_elem = x[b_path[1][0]][b_path[1][1]]
    min_elem_coord = (b_path[1][0], b_path[1][1])
    for i in range(len(b_path)):
        if signs[i] == '-':
            if x[b_path[i][0]][b_path[i][1]] < min_cycle_elem:
                min_cycle_elem = x[b_path[i][0]][b_path[i][1]]
                min_elem_coord = (b_path[1][0], b_path[1][1])
                
    #Шаг 10 ==========================================      
    for i in range(len(b_path)):
        if signs[i] == '-':
            x[b_path[i][0]][b_path[i][1]] = x[b_path[i][0]][b_path[i][1]] - min_cycle_elem
        elif signs[i] == '+':
            x[b_path[i][0]][b_path[i][1]] = x[b_path[i][0]][b_path[i][1]] + min_cycle_elem

    b_path.remove(min_elem_coord)

Оптимальный план: 
[[  0.   0. 100.]
 [  0. 200. 100.]
 [300.   0.   0.]]
